<a href="https://colab.research.google.com/github/LCaravaggio/AnalisisPredictivo/blob/master/04_%C3%81rboles/%C3%81rbol%20Overtited.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cargar datos

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import json

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

drive.mount('/content/drive', force_remount=True)
with open("/content/drive/My Drive/kaggle.json", 'r') as f:
    api_token= json.load(f)

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d mohidabdulrehman/ultimate-car-price-prediction-dataset

Mounted at /content/drive
Mounted at /content/drive
  0% 0.00/665k [00:00<?, ?B/s]
100% 665k/665k [00:00<00:00, 101MB/s]


In [3]:
import zipfile
import os

os.listdir()

for file in os.listdir():
    if file.endswith('.zip'):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()

# Preprar datos

In [4]:
df = pd.read_csv('Clean Data_pakwheels.csv', index_col=0)
df.head()

,Company Name,Model Name,Price,Model Year,Location,Mileage,Engine Type,Engine Capacity,Color,Assembly,Body Type,Transmission Type,Registration Status
0,Toyota,Vitz,2385000,2017,Islamabad,9869,Petrol,1000,Silver,Imported,Hatchback,Automatic,Un-Registered
1,Toyota,Corolla,111000,2019,KPK,11111,Petrol,1300,White,Local,Sedan,Automatic,Registered
2,Suzuki,Alto,1530000,2019,KPK,17500,Petrol,660,White,Local,Hatchback,Automatic,Un-Registered
3,Suzuki,Alto,1650000,2019,Punjab,9600,Petrol,660,White,Local,Hatchback,Manual,Registered
4,Toyota,Corolla,1435000,2010,Islamabad,120000,Petrol,1300,Black,Local,Sedan,Manual,Registered


In [16]:
df.shape

(46022, 13)

In [5]:
y = df['Price']
X = df.drop(['Price'], axis=1)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46022 entries, 0 to 46022
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Company Name         46022 non-null  object
 1   Model Name           46022 non-null  object
 2   Price                46022 non-null  int64 
 3   Model Year           46022 non-null  int64 
 4   Location             46022 non-null  object
 5   Mileage              46022 non-null  int64 
 6   Engine Type          46022 non-null  object
 7   Engine Capacity      46022 non-null  int64 
 8   Color                46022 non-null  object
 9   Assembly             46022 non-null  object
 10  Body Type            46022 non-null  object
 11  Transmission Type    46022 non-null  object
 12  Registration Status  46022 non-null  object
dtypes: int64(4), object(9)
memory usage: 4.9+ MB


In [8]:
cat_cols = ['Company Name', 'Model Name', 'Location', 'Engine Type',
            'Color', 'Assembly', 'Body Type', 'Transmission Type', 'Registration Status']
num_cols=['Mileage', 'Engine Capacity', 'Model Year']

In [14]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
X_encoded=enc.fit_transform(X[cat_cols])

In [17]:
from scipy.sparse import hstack
X_total = hstack((X_encoded, X[num_cols]))

# Un árbol

In [24]:
from sklearn.tree import DecisionTreeRegressor
reg = DecisionTreeRegressor()
reg.fit(X_total, y)

DecisionTreeRegressor()

In [25]:
reg.score(X_total, y)

0.9996393279175565

# Partir los datos

In [28]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_total, y, test_size=0.3, random_state=0)

In [29]:
reg.fit(x_train, y_train)

DecisionTreeRegressor()

In [30]:
reg.score(x_train, y_train)

0.9998191935105722

In [31]:
reg.score(x_test, y_test)

0.9434193291547721

# Poda

In [36]:
reg = DecisionTreeRegressor(max_depth=10)
reg.fit(x_train, y_train)
print(reg.score(x_train, y_train))
print(reg.score(x_test, y_test))

0.9784282923295843
0.9531784082492162


# XGBoost

In [37]:
from xgboost import XGBRegressor
xgb_reg=XGBRegressor()
xgb_reg.fit(x_train, y_train)
print(xgb_reg.score(x_train, y_train))
print(xgb_reg.score(x_test, y_test))

0.9910379302027627
0.9695308420395181
